# Ejemplo Práctico: Análisis de Transacciones con Tarjeta de Crédito usando RPy2
----
Este ejemplo demuestra cómo usar **RPy2** para combinar:
- Manipulación de datos en Python (pandas)
- Análisis estadístico avanzado en R
- Visualización especializada con ggplot2

**Caso de uso:** Detección de patrones fraudulentos y análisis de comportamiento de compras.

¿Qué hace este análisis de transacciones?
Este Jupyter Notebook es una herramienta que simula y analiza el comportamiento de transacciones con tarjeta de crédito. Su objetivo principal es entender patrones de compra y detectar posibles fraudes, combinando las fortalezas de dos lenguajes de programación: Python y R.

Piensa en él como un análista de fraude que examina miles de compras para encontrar lo que es normal y lo que podría ser sospechoso.

¿Cómo funciona el "análista" de transacciones?
El proceso se divide en cinco pasos principales, cada uno construido sobre el anterior:

Generación de Datos de Prueba (Simulación de Compras)
¿Qué hace? Crea un conjunto grande y falso de transacciones de tarjeta de crédito, como si miles de personas estuvieran comprando en diferentes lugares y momentos.

¿Por qué es útil? Nos permite practicar el análisis sin usar datos reales y sensibles. Incluye algunos "fraudes" simulados para que el detective tenga algo que buscar.

¿Cómo lo hace? Utiliza la librería Python (Pandas) para construir estas transacciones con detalles como el monto, la fecha, la categoría de la tienda, la ciudad, e incluso si es un fraude (conocido solo por el script en este punto).
Análisis Básico con Python

¿Qué hace? Realiza un primer vistazo rápido a los datos de las transacciones.

¿Por qué es útil? Nos da una idea general de cómo se distribuyen los montos, cuáles son las categorías de gasto más comunes y cuánto dinero se mueve en cada una. También identifica transacciones inusualmente grandes.

¿Cómo lo hace? Emplea las capacidades de Python (Pandas) para calcular estadísticas como promedios, sumas y el número de transacciones por tipo de tienda.

Análisis Estadístico Avanzado con R

¿Qué hace? Suma las capacidades de un experto en estadísticas (R) para profundizar en los patrones de los datos.

¿Por qué es útil?
Entiende las distribuciones: Nos dice si los montos de compra siguen un patrón predecible o si son muy variados.
Compara categorías: Determina si el gasto promedio es significativamente diferente entre categorías (ej., ¿se gasta mucho más en electrónicos que en farmacia?).

Predice Fraudes: Intenta construir un "modelo" que aprenda qué características (como el monto o la hora) hacen que una transacción sea más propensa a ser fraudulenta.

Analiza el tiempo: Busca tendencias y ciclos en el total de transacciones a lo largo del tiempo (ej., ¿más compras los fines de semana o a fin de mes?).

¿Cómo lo hace? Usa RPy2 para enviar los datos de Python a R, donde se aplican métodos estadísticos avanzados.
Visualización Profesional con R (ggplot2)

¿Qué hace? Crea gráficos claros y atractivos que nos ayudan a "ver" los patrones y resultados del análisis.

¿Por qué es útil? Es mucho más fácil entender las relaciones en los datos con un buen gráfico que leyendo números.

¿Cómo lo hace? Utiliza RPy2 para enviar los datos a R y luego aprovecha ggplot2, una de las librerías de gráficos más potentes de R, para dibujar:
Gráficos que muestran cómo varían los montos en cada categoría de compra.
Gráficos que siguen la cantidad de transacciones (normales y fraudulentas) a lo largo de las semanas.
"Mapas de calor" que muestran cuándo (qué día y hora) hay más actividad de transacciones.
Comparación de Rendimiento

¿Qué hace? Mide qué tan rápido realizan algunas operaciones estadísticas Python y R.

¿Por qué es útil? Aunque ambos son muy eficientes, esta comparación nos da una idea de cuándo uno podría ser ligeramente mejor que el otro para ciertas tareas, incluso considerando la comunicación entre ellos.

¿Cómo lo hace? Ejecuta cálculos básicos (como la media y la desviación estándar) en Python (NumPy) y luego los repite en R para comparar los tiempos.
En resumen, este script actúa como un centro de comando donde Python organiza los datos y las instrucciones, y R entra en acción cuando se necesita un análisis estadístico profundo o gráficos especializados, todo para ayudarnos a descubrir secretos ocultos en el comportamiento de las transacciones.









### Importar librerías de Python
Importamos las librerías de Python necesarias para la manipulación de datos y la generación de datos ficticios.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import time

----
## Configuración de RPy2
----
Configuramos RPy2 para permitir la interacción entre Python y R. Esto incluye la activación de conversiones automáticas de tipos de datos y la importación de paquetes R.

### Importar módulos de RPy2
Estos módulos son esenciales para la comunicación entre Python y R.

In [2]:
import os
import sys # Importar sys para salir si hay problemas con R_HOME

# --- INICIA EL TROUBLESHOOTING / CONFIGURACIÓN ---
# 1. Define la ruta RAÍZ de tu instalación de R.
#    ¡Usamos la ruta corta que R.home() te devolvió, que es más segura!
#    'C:/PROGRA~1/R/R-42~1.2' es equivalente a 'C:\\Program Files\\R\\R-4.2.2'
r_home_path = 'C:\\PROGRA~1\\R\\R-42~1.2' 

# 2. Establece la variable de entorno R_HOME.
#    RPy2 usa R_HOME para encontrar la instalación base de R.
os.environ['R_HOME'] = r_home_path
os.environ['LANG'] = 'en_US.UTF-8'

# 3. Añade la ruta del ejecutable de R (bin/x64 para 64-bit) a la variable PATH.
#    Esto permite que Python (y rpy2) encuentren el ejecutable 'R.exe'.
r_bin_path = os.path.join(r_home_path, 'bin', 'x64') # Para R de 64 bits (lo más probable)

# Añade esta ruta al PATH existente si aún no está.
# Es crucial añadirla al inicio del PATH para que se encuentre antes que otras posibles versiones de R o herramientas.
if r_bin_path not in os.environ['PATH']:
    os.environ['PATH'] = r_bin_path + os.pathsep + os.environ['PATH']

# --- AÑADIMOS ESTO PARA DIAGNÓSTICO DE RTOOLS ---
# Encuentra la ruta donde RTools debería estar instalado.
# Para RTools42, suele ser C:\rtools42\usr\bin o similar.
# Puedes verificar la ruta exacta en la instalación de RTools.
rtools_path = 'C:\\rtools42\\usr\\bin' # <--- ¡VERIFICA ESTA RUTA DE TU INSTALACIÓN DE RTOOLS!
# Si no instalaste en C:\rtools42, ajusta esta línea.

# Añade la ruta de RTools al PATH si no está ya.
# Es vital que esta ruta esté en el PATH para que 'sh.exe' y otras herramientas sean encontradas.
if rtools_path not in os.environ['PATH']:
    os.environ['PATH'] = rtools_path + os.pathsep + os.environ['PATH']

print("Verificando PATH después de añadir R y RTools:")
print(os.environ['PATH'])

# --- TERMINA EL TROUBLESHOOTING / CONFIGURACIÓN ---

# Importar las librerías necesarias de Python
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Configuración de RPy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri, numpy2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

# REMOVE THESE LINES, THEY ARE DEPRECATED
# pandas2ri.activate()
# numpy2ri.activate()

# Importar paquetes R necesarios
print("\nImportando paquetes R...")
try:
    # Set the conversion context explicitly here using localconverter
    # This replaces pandas2ri.activate() and numpy2ri.activate()
    with localconverter(ro.default_converter + pandas2ri.converter + numpy2ri.converter) as cv:
        base = importr('base')
        stats = importr('stats')
        utils = importr('utils')
        
        r_packages = ['ggplot2', 'dplyr', 'lubridate', 'plotly']
        for pkg in r_packages:
            try:
                importr(pkg)
                print(f"  {pkg} cargado correctamente")
            except Exception as pkg_e:
                print(f"  Error al cargar {pkg}: {pkg_e}")
                print(f"  Intentando instalar {pkg}...")
                utils.install_packages(pkg)
                print(f"  {pkg} instalado y cargado")
                
        ggplot2 = importr('ggplot2')
        dplyr = importr('dplyr')
        lubridate = importr('lubridate')
        
except Exception as e:
    print(f"\n¡ERROR CRÍTICO! RPy2 o R no pudieron inicializarse correctamente: {e}")
    print("Por favor, revisa los pasos de instalación de R y RTools, y la configuración de PATH.")
    sys.exit(1) # Salir del script si hay un error fatal con R.

Verificando PATH después de añadir R y RTools:
C:\rtools42\usr\bin;C:\PROGRA~1\R\R-42~1.2\bin\x64;C:\virtual_environment\bootcamp03\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files\Microsoft\jdk-11.0.27.6-hotspot\bin;C:\Program Files (x86)\VMware\VMware Player\bin\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\170\Tools\Binn\;C:\Program Files (x86)\Microsoft SQL Server\150\Tools\Binn\;C:\Program Files\Microsoft SQL Server\150\Tools\Binn\;C:\Program Files\Microsoft SQL Server\150\DTS\Binn\;C:\Program Files (x86)\Microsoft SQL Server\150\DTS\Binn\;C:\Program Files\Azure Data Studio\bin;C:\Program Files\PuTTY\;C:\Program Files\dotnet\;C:\Program Files\Git\cmd;C:\Program Files (x86)\Google\Cloud SDK\google-cloud-sdk\bin;C:\Program Files\Snowflake SnowSQL\;C:\Program

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.
C:\virtual_environment\bootcamp03\lib\site-packages\rpy2\rinterface\__init__.py:1211: UserWarning: Environment variable "PATH" redefined by R and overriding existing variable. Current: "C:\rtools42\usr\bin;C:\PROGRA~1\R\R-42~1.2\bin\x64;C:\virtual_environment\bootcamp03\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files\Microsoft\jdk-11.0.27.6-hotspot\bin;C:\Program Files (x86)\VMware\VMware Player\bin\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\170\Tools\Binn\;C:\Program Files (x86)\Microsoft SQL Server\150\Tools\Binn\;C:\Program Files\Microsoft SQL Server\150\Tools\Binn\;C:\Program Files\Microsoft SQL Server\150\DTS\Binn\;C:\Program Files (x86)


Importando paquetes R...
  ggplot2 cargado correctamente
  dplyr cargado correctamente
  lubridate cargado correctamente


R callback write-console: Warning:  
R callback write-console:  package 'plotly' is in use and will not be installed
  


  Error al cargar plotly: list index out of range
  Intentando instalar plotly...
  plotly instalado y cargado


----
## 1. Generación de Datos Ficticios
----
Creamos un conjunto de datos simulado de transacciones con tarjeta de crédito para trabajar. Este dataset incluirá información como ID de usuario, fecha y hora, monto, categoría de comercio, ciudad, estado de la transacción y una etiqueta para identificar transacciones fraudulentas.

In [3]:
def generar_transacciones_ficticias(n_transacciones=5000):
    """
    Genera un dataset ficticio de transacciones con tarjeta de crédito
    """
    print(f"Generando {n_transacciones} transacciones ficticias...")
    
    np.random.seed(42)
    random.seed(42)
    
    n_usuarios = 500
    inicio_fecha = datetime(2024, 1, 1)
    fin_fecha = datetime(2024, 12, 31)
    
    categorias = [
        'Supermercado', 'Gasolina', 'Restaurante', 'Ropa', 'Farmacia',
        'Electrónicos', 'Online', 'Entretenimiento', 'Salud', 'Educación'
    ]
    
    transacciones = []
    
    for _ in range(n_transacciones):
        usuario_id = f"USER_{random.randint(1000, 1000 + n_usuarios):04d}"
        
        dias_diff = (fin_fecha - inicio_fecha).days
        fecha_random = inicio_fecha + timedelta(days=random.randint(0, dias_diff))
        
        # Hora del día (patrones realistas)
        if random.random() < 0.7:  # 70% durante horas normales (6 AM - 10 PM)
            hora = random.randint(6, 22)
        else:  # 30% durante horas inusuales
            # Generar horas entre 23:00 y 05:00
            # Opción más robusta: elegir de una lista predefinida de horas inusuales
            # o generar en dos rangos separados
            hora = random.choice([23, 0, 1, 2, 3, 4, 5]) 
            # Si quieres asignar pesos diferentes, puedes usar random.choices
            # Por ejemplo, para que las horas 0-5 sean más comunes que las 23:00 en este segmento
            # hora = random.choices([random.randint(23, 23), random.randint(0, 5)], weights=[1, 6])[0]


        fecha_completa = fecha_random.replace(
            hour=hora, 
            minute=random.randint(0, 59),
            second=random.randint(0, 59)
        )
        
        categoria = random.choice(categorias)
        
        montos_base = {
            'Supermercado': (20, 150),
            'Gasolina': (30, 80),
            'Restaurante': (15, 120),
            'Ropa': (25, 300),
            'Farmacia': (10, 60),
            'Electrónicos': (50, 1500),
            'Online': (10, 200),
            'Entretenimiento': (20, 100),
            'Salud': (50, 500),
            'Educación': (100, 800)
        }
        
        min_monto, max_monto = montos_base[categoria]
        monto = round(np.random.gamma(2, (max_monto - min_monto) / 6) + min_monto, 2)
        
        ciudades = ['Madrid', 'Barcelona', 'Valencia', 'Sevilla', 'Bilbao', 
                    'Málaga', 'Zaragoza', 'Murcia', 'Palma', 'Las Palmas']
        ciudad = random.choice(ciudades)
        
        es_fraude = random.random() < 0.02
        
        if es_fraude:
            monto *= random.uniform(3, 10)
            if random.random() < 0.5:
                # Horas inusuales para fraude
                # Asegurarse de que el rango de horas de fraude también sea válido
                hora_fraude = random.choice([1, 2, 3, 4]) # Más probable en madrugada profunda
                fecha_completa = fecha_completa.replace(hour=hora_fraude)
        
        if es_fraude and random.random() < 0.3:
            estado = 'Rechazada'
        else:
            estado = random.choices(
                ['Aprobada', 'Rechazada', 'Pendiente'],
                weights=[0.92, 0.06, 0.02]
            )[0]
        
        transacciones.append({
            'usuario_id': usuario_id,
            'fecha_hora': fecha_completa,
            'monto': monto,
            'categoria': categoria,
            'ciudad': ciudad,
            'estado': estado,
            'es_fraude': es_fraude,
            'dia_semana': fecha_completa.strftime('%A'),
            'hora': fecha_completa.hour,
            'mes': fecha_completa.month
        })
    
    df = pd.DataFrame(transacciones)
    print(f"Dataset generado: {len(df)} transacciones")
    print(f"Transacciones fraudulentas: {df['es_fraude'].sum()} ({df['es_fraude'].mean()*100:.1f}%)")
    
    return df

----
## 2. Análisis con Python (Pandas)
----
Realizamos un análisis exploratorio de datos inicial utilizando las capacidades de manipulación de DataFrames de **pandas**. Esto incluye estadísticas descriptivas y agrupaciones por categorías.

In [4]:
def analisis_python(df):
    """
    Realiza un análisis básico del dataset de transacciones utilizando la librería pandas de Python.
    Incluye estadísticas descriptivas y una detección simple de anomalías.
    
    Args:
        df (pd.DataFrame): El DataFrame de pandas que contiene los datos de transacciones.
        
    Returns:
        pd.DataFrame: El mismo DataFrame de entrada (no se modifica para el retorno en este caso).
    """
    print("\nANÁLISIS CON PYTHON (pandas)")
    print("=" * 50)
    
    # `df['monto'].describe()` genera estadísticas descriptivas básicas para la columna 'monto',
    # incluyendo conteo, media, desviación estándar, valores mínimo y máximo, y cuartiles.
    print("Estadísticas Básicas del Monto:")
    print(df['monto'].describe())
    
    # `groupby('categoria')` agrupa el DataFrame por la columna 'categoria'.
    # `.agg()` aplica múltiples funciones de agregación a las columnas especificadas:
    # - 'monto': se cuenta el número de transacciones (`'count'`), se calcula el monto promedio (`'mean'`)
    #   y el monto total (`'sum'`) por categoría.
    # - 'es_fraude': se suma para obtener el número total de transacciones fraudulentas por categoría
    #   (ya que `True` se considera 1 y `False` como 0).
    # `.round(2)` redondea los resultados a dos decimales.
    print("\nAnálisis por Categoría:")
    categoria_stats = df.groupby('categoria').agg({
        'monto': ['count', 'mean', 'sum'],  
        'es_fraude': 'sum'                 
    }).round(2)
    print(categoria_stats)
    
    # Detección básica de anomalías basada en el monto de la transacción.
    # Se calcula el percentil 99 de la columna 'monto'. Esto significa que el 99% de las transacciones
    # tienen un monto igual o inferior a este valor.
    print("\nDetección Básica de Anomalías:")
    q99 = df['monto'].quantile(0.99)  
    # Se filtran las transacciones cuyo monto es estrictamente mayor que el percentil 99.
    # Estas transacciones se consideran anomalías o valores atípicos en términos de monto.
    anomalias = df[df['monto'] > q99] 
    print(f"Transacciones con monto > P99 (EUR{q99:.2f}): {len(anomalias)}")
    
    return df

----
## 3. Análisis Estadístico Avanzado con R
----
Aquí explotamos el poder de R para análisis estadísticos más complejos, como tests de normalidad, ANOVA, modelos de regresión logística y análisis de series temporales.

In [5]:
def analisis_estadistico_r(df):
    """
    Realiza análisis estadísticos avanzados utilizando las capacidades de R a través de RPy2.
    Incluye tests de normalidad, ANOVA para comparar grupos, un modelo de regresión logística
    para la detección de fraude, y un análisis de series temporales.
    
    Args:
        df (pd.DataFrame): El DataFrame de pandas que contiene los datos de transacciones.
    """
    print("\nANÁLISIS ESTADÍSTICO AVANZADO CON R")
    print("=" * 50)
    
    try:
        # Se utiliza `localconverter(pandas2ri.converter)` para asegurar que el DataFrame de pandas
        # se convierte correctamente a un DataFrame de R dentro de este bloque.
        with localconverter(pandas2ri.converter):
            df_r = pandas2ri.py2rpy(df)
            # El DataFrame de R se asigna a una variable global en el entorno de R (`transacciones`).
            # Esto permite que los scripts de R subsiguientes accedan a los datos fácilmente.
            ro.globalenv['transacciones'] = df_r
        
        # ### 1. Análisis de distribución con tests estadísticos
        # Se realizan tests de normalidad para evaluar si la distribución de los montos de las transacciones
        # se aproxima a una distribución normal.
        print("Tests de Normalidad y Distribución...")
        
        # Se ejecuta un bloque de código R directamente utilizando `ro.r()`.
        # - `sample(transacciones$monto, 1000)`: Toma una muestra aleatoria de 1000 montos. El test de Shapiro-Wilk es más fiable con muestras más pequeñas.
        # - `shapiro.test()`: Realiza el test de normalidad de Shapiro-Wilk. Un p-value > 0.05 sugiere que los datos son normalmente distribuidos.
        # - `ks.test()`: Realiza el test de Kolmogorov-Smirnov para comparar la distribución de la muestra con una distribución normal teórica.
        # Se devuelve una lista de resultados desde R a Python.
        resultado_shapiro = ro.r('''
            # Test de normalidad Shapiro-Wilk (apropiado para muestras pequeñas a moderadas)
            muestra_monto <- sample(transacciones$monto, 1000)
            shapiro_test <- shapiro.test(muestra_monto)
            
            # Test de Kolmogorov-Smirnov para comparar con una distribución normal
            # Se compara la distribución de la muestra con una distribución normal teórica
            # que tiene la misma media y desviación estándar que la muestra.
            ks_test <- ks.test(muestra_monto, "pnorm", 
                               mean = mean(muestra_monto), 
                               sd = sd(muestra_monto))
            
            # Se devuelven los p-valores de ambos tests y una bandera booleana
            # que indica si la distribución puede considerarse normal basándose en Shapiro-Wilk (p > 0.05).
            list(
                shapiro_p = shapiro_test$p.value,
                ks_p = ks_test$p.value,
                distribucion_normal = shapiro_test$p.value > 0.05
            )
        ''')
        
        # Se imprimen los p-valores obtenidos y la conclusión sobre la normalidad de la distribución.
        print(f"  Shapiro-Wilk p-value: {resultado_shapiro[0][0]:.6f}")
        print(f"  Kolmogorov-Smirnov p-value: {resultado_shapiro[1][0]:.6f}")
        print(f"  ¿Distribución normal?: {'Sí' if resultado_shapiro[2][0] else 'No'}")
        
        # ### 2. Análisis de varianza (ANOVA) por categoría
        # El ANOVA se utiliza para determinar si existen diferencias estadísticamente significativas
        # en las medias de los montos de las transacciones entre las diferentes categorías de comercio.
        print("\nAnálisis de Varianza (ANOVA) por Categoría...")
        
        anova_resultado = ro.r('''
            # `aov(monto ~ categoria, data = transacciones)`: Realiza el análisis de varianza.
            # `monto` es la variable dependiente y `categoria` es el factor de agrupación.
            anova_resultado <- aov(monto ~ categoria, data = transacciones)
            # `summary()`: Proporciona un resumen detallado de los resultados del ANOVA, incluyendo el estadístico F y el p-value.
            anova_summary <- summary(anova_resultado)
            
            # `TukeyHSD()`: Realiza un test post-hoc de Tukey HSD (Honest Significant Difference).
            # Este test se usa cuando el ANOVA es significativo para identificar qué pares de categorías específicas
            # tienen diferencias de medias significativas, controlando la tasa de error por comparaciones múltiples.
            tukey_resultado <- TukeyHSD(anova_resultado)
            
            # Se devuelven el estadístico F, el p-value y una bandera booleana de significancia.
            list(
                f_statistic = anova_summary[[1]][1,4], # Estadístico F del ANOVA
                p_value = anova_summary[[1]][1,5],     # P-value del ANOVA
                significativo = anova_summary[[1]][1,5] < 0.05 # True si el p-value es menor que 0.05
            )
        ''')
        
        # Se imprimen los resultados del ANOVA.
        print(f"  F-statistic: {anova_resultado[0][0]:.4f}")
        print(f"  P-value: {anova_resultado[1][0]:.6f}")
        print(f"  ¿Diferencias significativas?: {'Sí' if anova_resultado[2][0] else 'No'}")
        
        # ### 3. Modelo de regresión logística para detección de fraude
        # Se construye un modelo de regresión logística para predecir la probabilidad de que una transacción
        # sea fraudulenta (`es_fraude`) basándose en `monto`, `hora` y `categoria`.
        print("\nModelo de Regresión Logística para Detección de Fraude...")
        
        modelo_fraude = ro.r('''
            # Preparación de datos: Se crea una nueva columna `hora_cat` dividiendo la hora en intervalos.
            # Esto puede ayudar al modelo a capturar patrones relacionados con franjas horarias específicas.
            transacciones$hora_cat <- cut(transacciones$hora, 
                                          breaks = c(0, 6, 12, 18, 24),
                                          labels = c("Madrugada", "Mañana", "Tarde", "Noche"),
                                          include.lowest = TRUE)
            
            # `glm()`: Función para ajustar modelos lineales generalizados.
            # `es_fraude ~ monto + hora + categoria`: Fórmula del modelo, donde `es_fraude` es la variable dependiente (binaria).
            # `family = binomial()`: Especifica que es un modelo de regresión logística (para respuestas binarias).
            modelo <- glm(es_fraude ~ monto + hora + categoria, 
                          data = transacciones, 
                          family = binomial())
            
            # `summary(modelo)`: Proporciona un resumen detallado del modelo, incluyendo coeficientes, errores estándar, etc.
            modelo_summary <- summary(modelo)
            
            # Predicciones: Se obtienen las probabilidades predichas por el modelo (`type = "response"`).
            predicciones <- predict(modelo, type = "response")
            # Se convierten las probabilidades en clases binarias (TRUE/FALSE) utilizando un umbral de 0.5.
            pred_clase <- ifelse(predicciones > 0.5, TRUE, FALSE)
            # `table()`: Crea una matriz de confusión comparando las etiquetas reales con las predichas.
            matriz_confusion <- table(transacciones$es_fraude, pred_clase)
            
            # Cálculo de métricas de evaluación del modelo:
            # - Precisión: TP / (TP + FP). Proporción de predicciones positivas correctas.
            precision <- matriz_confusion[2,2] / sum(matriz_confusion[,2])
            # - Recall (Sensibilidad): TP / (TP + FN). Proporción de casos positivos reales identificados correctamente.
            recall <- matriz_confusion[2,2] / sum(matriz_confusion[2,])
            # - F1-Score: Media armónica de Precisión y Recall. Útil cuando hay un desequilibrio de clases.
            f1_score <- 2 * (precision * recall) / (precision + recall)
            
            # Se devuelve el AIC (Criterio de Información de Akaike), las métricas de evaluación y los coeficientes del modelo.
            list(
                aic = modelo$aic,
                precision = precision,
                recall = recall,
                f1_score = f1_score,
                coeficientes = modelo$coefficients
            )
        ''')
        
        # Se imprimen las métricas de evaluación del modelo de fraude.
        print(f"  AIC del modelo: {modelo_fraude[0][0]:.2f}")
        print(f"  Precision: {modelo_fraude[1][0]:.4f}")
        print(f"  Recall: {modelo_fraude[2][0]:.4f}")
        print(f"  F1-Score: {modelo_fraude[3][0]:.4f}")
        
        # ### 4. Análisis de series temporales
        # Se analiza el patrón de los montos de las transacciones a lo largo del tiempo para identificar tendencias y estacionalidades.
        print("\nAnálisis de Series Temporales...")
        
        serie_temporal = ro.r('''
            library(lubridate) # Se carga la librería `lubridate` para facilitar la manipulación de fechas.
            
            # Se convierte la columna `fecha_hora` a un formato de solo fecha (`as.Date`).
            transacciones$fecha <- as.Date(transacciones$fecha_hora)
            # Se agrupan los datos por `fecha` y se suman los `monto` para obtener el total diario.
            serie_diaria <- aggregate(monto ~ fecha, data = transacciones, sum)
            
            # `ts()`: Crea un objeto de serie temporal en R.
            # `serie_diaria$monto`: Los datos numéricos de la serie temporal.
            # `frequency = 7`: Indica que la serie tiene una frecuencia semanal (7 observaciones por ciclo).
            ts_datos <- ts(serie_diaria$monto, frequency = 7) 
            
            # `decompose()`: Descompone la serie temporal en sus componentes de tendencia, estacionalidad y residuo.
            # Esto ayuda a visualizar y entender los patrones a largo plazo y los ciclos repetitivos.
            decomp <- decompose(ts_datos)
            
            # Se calcula la media de la componente de tendencia y la varianza de la componente estacional.
            # `na.rm = TRUE` para ignorar los valores `NA` (not available).
            tendencia_media <- mean(decomp$trend, na.rm = TRUE)
            estacionalidad_var <- var(decomp$seasonal, na.rm = TRUE)
            
            # Se devuelven las estadísticas calculadas y el número de observaciones en la serie temporal.
            list(
                tendencia_media = tendencia_media,
                varianza_estacional = estacionalidad_var,
                observaciones = length(ts_datos)
            )
        ''')
        
        # Se imprimen los resultados del análisis de series temporales.
        print(f"  Tendencia media: EUR{serie_temporal[0][0]:.2f}")
        print(f"  Varianza estacional: {serie_temporal[1][0]:.2f}")
        print(f"  Observaciones: {int(serie_temporal[2][0])}")
        
    except Exception as e:
        # Manejo de errores específicos para el análisis en R.
        print(f"Error en análisis R: {e}")
        print("Verifica que R y los paquetes requeridos estén instalados y funcionando correctamente.")

----
## 4. Visualización Avanzada con GGPlot2
----
Generamos gráficos de alta calidad utilizando la librería **ggplot2** de R, conocida por su capacidad para crear visualizaciones estéticas y personalizables. Los gráficos se guardarán como archivos PNG.

In [6]:
def visualizacion_r(df):
    """
    Crea visualizaciones avanzadas utilizando la poderosa librería ggplot2 de R.
    Los gráficos generados se guardan como archivos PNG en el directorio de trabajo actual.
    
    Args:
        df (pd.DataFrame): El DataFrame de pandas que contiene los datos de transacciones.
    """
    print("\nVISUALIZACIÓN AVANZADA CON GGPLOT2")
    print("=" * 50)
    
    try:
        # Se convierte el DataFrame de pandas a un DataFrame de R para que ggplot2 pueda trabajar con él.
        with localconverter(pandas2ri.converter):
            df_r = pandas2ri.py2rpy(df)
            ro.globalenv['datos_viz'] = df_r # Se asigna a una variable global en R para su uso en los scripts R.
        
        # ### 1. Gráfico de distribución de montos por categoría
        # Se crea un boxplot para visualizar la distribución de los montos de transacciones
        # para cada categoría de comercio. Se añaden puntos individuales para mostrar la densidad de datos.
        print("Creando gráfico de distribución por categoría...")
        
        ro.r('''
            library(ggplot2) # Se carga la librería ggplot2 para gráficos.
            library(dplyr)   # Se carga la librería dplyr para manipulación de datos (piping).
            
            # `ggplot()`: Inicializa el objeto gráfico, especificando el DataFrame (`datos_viz`) y las estéticas (`aes`).
            # `x = categoria`: La variable en el eje X será la categoría.
            # `y = monto`: La variable en el eje Y será el monto.
            # `fill = categoria`: Rellena las cajas y puntos según la categoría.
            p1 <- ggplot(datos_viz, aes(x = categoria, y = monto, fill = categoria)) +
                geom_boxplot() + # Añade una capa de boxplots para mostrar la distribución.
                # `geom_jitter()`: Añade puntos individuales con un poco de "ruido" para evitar superposición y mostrar la densidad real.
                # `alpha = 0.3`: Transparencia de los puntos.
                # `width = 0.2`: Ancho del jitter.
                # `size = 0.5`: Tamaño de los puntos.
                geom_jitter(alpha = 0.3, width = 0.2, size = 0.5) +
                theme_minimal() + # Aplica un tema minimalista al gráfico.
                # `theme(axis.text.x = element_text(angle = 45, hjust = 1))`: Rota las etiquetas del eje X para mejor legibilidad.
                theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
                # `labs()`: Define los títulos y etiquetas del gráfico.
                labs(title = "Distribución de Montos por Categoría",
                     subtitle = "Análisis de Transacciones con Tarjeta de Crédito",
                     x = "Categoría", y = "Monto (EUR)",
                     fill = "Categoría") +
                scale_fill_viridis_d() # Aplica una paleta de colores discreta de la familia Viridis.
            
            # `ggsave()`: Guarda el gráfico como un archivo de imagen PNG.
            # `width` y `height`: Dimensiones del gráfico en pulgadas.
            # `dpi`: Resolución en puntos por pulgada.
            ggsave("distribucion_categorias.png", p1, width = 12, height = 8, dpi = 300)
            print("Gráfico guardado: distribucion_categorias.png")
        ''')
        
        # ### 2. Análisis temporal de transacciones
        # Se crea un gráfico de líneas para mostrar la evolución semanal del número de transacciones,
        # diferenciando entre transacciones normales y fraudulentas.
        print("Creando gráfico de análisis temporal...")
        
        ro.r('''
            # Preparación de datos para el análisis temporal:
            datos_viz$fecha <- as.Date(datos_viz$fecha_hora) # Convierte la columna de fecha y hora a solo fecha.
            datos_viz$semana <- format(datos_viz$fecha, "%Y-%U") # Extrae el año y el número de semana (ej., "2024-01").
            
            # Se agrupan los datos por semana y por si la transacción es fraudulenta (`es_fraude`).
            # Luego se resume el número total de transacciones y el monto total para cada grupo.
            datos_semana <- datos_viz %>%
                group_by(semana, es_fraude) %>%
                summarise(
                    total_transacciones = n(),       # `n()` cuenta el número de filas en cada grupo.
                    monto_total = sum(monto),  # `sum(monto)` suma los montos en cada grupo.
                    .groups = 'drop'           # Elimina los grupos una vez finalizada la sumarización.
                )
            
            # Gráfico de series temporales:
            p2 <- ggplot(datos_semana, aes(x = semana, y = total_transacciones, 
                                           color = factor(es_fraude))) +
                geom_line(aes(group = factor(es_fraude)), linewidth = 1.2) + # Crea líneas conectando los puntos, agrupadas por `es_fraude`.
                geom_point(size = 2) + # Añade puntos en cada observación.
                theme_minimal() +
                theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
                labs(title = "Evolución Temporal de Transacciones",
                     subtitle = "Comparación entre Transacciones Normales y Fraudulentas",
                     x = "Semana", y = "Número de Transacciones",
                     color = "Tipo") +
                # `scale_color_manual()`: Asigna colores específicos a los valores de `es_fraude` (FALSE y TRUE).
                scale_color_manual(values = c("FALSE" = "#2E86AB", "TRUE" = "#F24236"),
                                   labels = c("Normal", "Fraude")) +
                # `facet_wrap()`: Divide el gráfico en paneles separados para cada valor de `es_fraude`.
                # `scales = "free_y"`: Permite que los ejes Y de cada panel tengan escalas independientes, útil si las magnitudes son muy diferentes.
                # `labeller`: Personaliza las etiquetas de los paneles.
                facet_wrap(~factor(es_fraude), scales = "free_y", 
                           labeller = labeller(.default = c("FALSE" = "Transacciones Normales",
                                                             "TRUE" = "Transacciones Fraudulentas")))
            
            ggsave("evolucion_temporal.png", p2, width = 14, height = 8, dpi = 300)
            print("Gráfico guardado: evolucion_temporal.png")
        ''')
        
        # ### 3. Heatmap de patrones por hora y día
        # Se crea un mapa de calor para visualizar la intensidad de las transacciones (número o monto)
        # en función de la hora del día y el día de la semana.
        print("Creando heatmap de patrones temporales...")
        
        ro.r('''
            # Preparación de datos para el heatmap:
            # Se convierte `dia_semana` a un factor numérico para asegurar un orden correcto en el eje Y del heatmap.
            datos_viz$dia_semana_num <- as.numeric(factor(datos_viz$dia_semana, 
                levels = c("Monday", "Tuesday", "Wednesday", "Thursday", 
                          "Friday", "Saturday", "Sunday")))
            
            # Se agrupan los datos por hora y día de la semana para calcular métricas agregadas.
            heatmap_data <- datos_viz %>%
                group_by(hora, dia_semana, dia_semana_num) %>%
                summarise(
                    num_transacciones = n(),       # Número de transacciones por cada celda (hora, día).
                    monto_promedio = mean(monto),  # Monto promedio por celda.
                    tasa_fraude = mean(as.numeric(es_fraude)), # Tasa de fraude por celda.
                    .groups = 'drop'
                )
            
            # Heatmap de actividad:
            p3 <- ggplot(heatmap_data, aes(x = hora, y = reorder(dia_semana, dia_semana_num), 
                                           fill = num_transacciones)) +
                geom_tile() + # `geom_tile()` crea las "baldosas" del heatmap, donde el color se mapea a `fill`.
                # `scale_fill_gradient()`: Define la escala de color para el relleno, de blanco (bajo) a azul oscuro (alto).
                scale_fill_gradient(low = "white", high = "darkblue", name = "Transacciones") +
                theme_minimal() +
                labs(title = "Patrón de Actividad: Transacciones por Hora y Día",
                     subtitle = "Heatmap de Intensidad de Transacciones",
                     x = "Hora del Día", y = "Día de la Semana") +
                # `scale_x_continuous()`: Ajusta las marcas del eje X para que aparezcan cada 2 horas (0, 2, 4, ..., 23).
                scale_x_continuous(breaks = seq(0, 23, 2))
            
            ggsave("heatmap_actividad.png", p3, width = 12, height = 6, dpi = 300)
            print("Gráfico guardado: heatmap_actividad.png")
        ''')
        
        print("\nTodas las visualizaciones han sido generadas exitosamente!")
        
    except Exception as e:
        # Manejo de errores específicos para la visualización en R.
        print(f"Error en visualización: {e}")

----
## 5. Comparación Python vs R (Rendimiento)
----
Realizamos una comparación de rendimiento entre operaciones estadísticas básicas ejecutadas en **Python (NumPy)** y en **R (a través de RPy2)**. Esto ayuda a entender el "overhead" de la interoperabilidad.

In [7]:
def comparacion_performance():
    """
    Compara el rendimiento de operaciones estadísticas básicas (media, desviación estándar, cuantiles)
    entre Python (NumPy) y R (a través de RPy2) utilizando un conjunto de datos grande.
    """
    print("\nCOMPARACIÓN DE PERFORMANCE PYTHON vs R")
    print("=" * 50)
    
    # Generar un conjunto de datos de prueba grande (100,000 puntos) con una distribución gamma.
    n_test = 100000
    datos_test = np.random.gamma(2, 1000, n_test)
    
    # --- Test de rendimiento en Python (NumPy) ---
    start_time = time.time() # Registra el tiempo de inicio antes de las operaciones en Python.
    media_py = np.mean(datos_test)      # Calcula la media del array.
    std_py = np.std(datos_test)         # Calcula la desviación estándar del array.
    # Calcula los percentiles 25, 50, 75, 95 y 99.
    quantiles_py = np.percentile(datos_test, [25, 50, 75, 95, 99])
    tiempo_python = time.time() - start_time # Calcula el tiempo transcurrido para las operaciones en Python.
    
    # --- Test de rendimiento en R (a través de RPy2) ---
    start_time = time.time() # Registra el tiempo de inicio antes de las operaciones en R.
    # Convierte el array de NumPy `datos_test` a un vector de tipo flotante de R (`ro.FloatVector`)
    # y lo asigna a una variable global en el entorno de R (`datos_test`).
    ro.globalenv['datos_test'] = ro.FloatVector(datos_test)
    
    # Ejecuta un bloque de código R para calcular las mismas estadísticas.
    resultado_r = ro.r('''
        media_r <- mean(datos_test) # Calcula la media en R.
        std_r <- sd(datos_test)     # Calcula la desviación estándar en R.
        # Calcula los cuantiles en R.
        quantiles_r <- quantile(datos_test, c(0.25, 0.5, 0.75, 0.95, 0.99))
        
        # Devuelve los resultados en una lista de R, que RPy2 convertirá de nuevo a un objeto Python.
        list(media = media_r, std = std_r, quantiles = quantiles_r)
    ''')
    tiempo_r = time.time() - start_time # Calcula el tiempo transcurrido para las operaciones en R.
    
    # Imprime los tiempos y los resultados de ambos entornos para la comparación.
    print(f"Python - Tiempo: {tiempo_python:.4f}s")
    print(f"    Media: {media_py:.2f}, Std: {std_py:.2f}")
    
    print(f"R - Tiempo: {tiempo_r:.4f}s")
    # Los resultados de R se acceden como elementos de la lista `resultado_r`.
    print(f"    Media: {resultado_r[0][0]:.2f}, Std: {resultado_r[1][0]:.2f}")
    
    # Calcula y muestra el ratio de velocidad.
    # Es importante señalar que el tiempo de R incluye el `overhead` de la comunicación con RPy2
    # y las conversiones de datos, lo que puede hacer que R parezca más lento para operaciones simples.
    print(f"\nRatio de velocidad (R / Python): {tiempo_r/tiempo_python:.2f}x")
    print("Nota: R incluye overhead de RPy2 y conversión de datos, lo que puede afectar la comparación directa.")

----
## Función Principal
----
Esta función coordina la ejecución de todos los pasos del análisis.

In [8]:
def main():
    """
    Función principal que orquesta la ejecución de todo el análisis de transacciones,
    demostrando la integración de Python y R a través de RPy2.
    """
    
    print("ANÁLISIS DE TRANSACCIONES CON TARJETA DE CRÉDITO")
    print("=" * 60)
    print("Demostrando el poder de RPy2 para integrar Python y R")
    print("=" * 60)
    
    # 1. Generar datos ficticios de transacciones.
    df = generar_transacciones_ficticias(5000)
    
    # --- Impresión de las primeras 10 filas del DataFrame ---
    print("\n--- Primeras 10 filas del DataFrame generado ---")
    print(df.head(10))
    print("--------------------------------------------------")
    # ---------------------------------------------------
    
    # 2. Realizar un análisis exploratorio básico utilizando pandas en Python.
    df = analisis_python(df)
    
    # 3. Ejecutar análisis estadístico avanzado utilizando el entorno de R.
    analisis_estadistico_r(df)
    
    # 4. Crear visualizaciones profesionales con ggplot2 en R.
    visualizacion_r(df)
    
    # 5. Comparar el rendimiento de operaciones entre Python y R.
    comparacion_performance()
    
    print("\n" + "=" * 60)
    print("RESUMEN DE VENTAJAS DE RPy2 DEMOSTRADAS:")
    print("=" * 60)
    print("- Manipulación de datos eficiente con pandas (Python)")
    print("- Tests estadísticos avanzados con R")
    print("- Visualizaciones profesionales con ggplot2")
    print("- Modelos estadísticos especializados (GLM)")
    print("- Análisis de series temporales")

In [9]:
# Este bloque asegura que la función `main()` se ejecute solo cuando el script se inicia directamente,
# y no cuando se importa como un módulo en otro script.
main()

ANÁLISIS DE TRANSACCIONES CON TARJETA DE CRÉDITO
Demostrando el poder de RPy2 para integrar Python y R
Generando 5000 transacciones ficticias...
Dataset generado: 5000 transacciones
Transacciones fraudulentas: 97 (1.9%)

--- Primeras 10 filas del DataFrame generado ---
  usuario_id          fecha_hora   monto     categoria      ciudad    estado  \
0  USER_1327 2024-02-27 14:15:14   56.89   Restaurante   Barcelona  Aprobada   
1  USER_1044 2024-10-29 06:05:13   93.50          Ropa       Palma  Aprobada   
2  USER_1366 2024-11-28 02:14:28  261.27     Educación      Bilbao  Aprobada   
3  USER_1412 2024-03-22 16:17:09   88.36          Ropa      Málaga  Aprobada   
4  USER_1183 2024-06-25 07:46:29  398.73         Salud   Barcelona  Aprobada   
5  USER_1282 2024-05-30 03:56:55  742.79  Electrónicos  Las Palmas  Aprobada   
6  USER_1338 2024-04-26 23:54:14   39.43      Gasolina    Zaragoza  Aprobada   
7  USER_1186 2024-03-24 12:42:17   50.58      Gasolina  Las Palmas  Aprobada   
8  USER_11

R callback write-console: In addition:   
R callback write-console: Warning message:
  
R callback write-console: In ks.test.default(muestra_monto, "pnorm", mean = mean(muestra_monto),  :  
R callback write-console: 
   
R callback write-console:  ties should not be present for the Kolmogorov-Smirnov test
  
R callback write-console: In addition:   
R callback write-console: Warning message:
  
R callback write-console: glm.fit: fitted probabilities numerically 0 or 1 occurred 
  


Tests de Normalidad y Distribución...
  Shapiro-Wilk p-value: 0.000000
  Kolmogorov-Smirnov p-value: 0.000000
  ¿Distribución normal?: No

Análisis de Varianza (ANOVA) por Categoría...
  F-statistic: 326.9964
  P-value: 0.000000
  ¿Diferencias significativas?: Sí

Modelo de Regresión Logística para Detección de Fraude...
  AIC del modelo: 364.16
  Precision: 0.8889
  Recall: 0.6598
  F1-Score: 0.7574

Análisis de Series Temporales...
  Tendencia media: EUR2331.30
  Varianza estacional: 20967.22
  Observaciones: 367

VISUALIZACIÓN AVANZADA CON GGPLOT2
Creando gráfico de distribución por categoría...
[1]

R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xe1 in position 4: invalid continuation byte <traceback object at 0x000001A0ADB8CA80>



Creando gráfico de análisis temporal...
[1]

R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xe1 in position 4: invalid continuation byte <traceback object at 0x000001A0ACA24140>



Creando heatmap de patrones temporales...
[1]

R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xe1 in position 4: invalid continuation byte <traceback object at 0x000001A0ACA25340>




Todas las visualizaciones han sido generadas exitosamente!

COMPARACIÓN DE PERFORMANCE PYTHON vs R
Python - Tiempo: 0.0030s
    Media: 2002.63, Std: 1417.99
R - Tiempo: 0.0050s
    Media: 2002.63, Std: 1418.00

Ratio de velocidad (R / Python): 1.67x
Nota: R incluye overhead de RPy2 y conversión de datos, lo que puede afectar la comparación directa.

RESUMEN DE VENTAJAS DE RPy2 DEMOSTRADAS:
- Manipulación de datos eficiente con pandas (Python)
- Tests estadísticos avanzados con R
- Visualizaciones profesionales con ggplot2
- Modelos estadísticos especializados (GLM)
- Análisis de series temporales


# Análisis de Métricas del Modelo

## 1. AIC del modelo: 364.16
**¿Qué es?**  
El AIC (Criterio de Información de Akaike) es como un "puntuaje de calidad" para tu modelo. Intenta estimar qué tan bien el modelo predice nuevos datos, penalizando la complejidad (más variables o reglas).

**Interpretación:**  
Un AIC más bajo generalmente indica un mejor modelo. No hay un "buen" o "mal" valor de AIC por sí solo; es más útil para comparar diferentes modelos que intentan resolver el mismo problema. En este caso, el valor 364.16 es una referencia interna de su rendimiento.

## 2. Precision: 0.8889
**¿Qué es?**  
La Precisión nos dice, de todas las transacciones que tu modelo marcó como fraudulentas, ¿cuántas fueron realmente fraudulentas?

**Interpretación:**  
Un valor de 0.8889 (o 88.89%) es muy bueno. Significa que, si tu modelo dice que una transacción es fraude, hay casi un 89% de posibilidades de que realmente lo sea. Esto es importante para evitar "falsas alarmas" que podrían molestar a los clientes.

## 3. Recall: 0.6598
**¿Qué es?**  
El Recall (también conocido como Sensibilidad) nos dice, de todas las transacciones que realmente fueron fraudulentas, ¿cuántas fue capaz de detectar tu modelo?

**Interpretación:**  
Un valor de 0.6598 (o 65.98%) significa que tu modelo logró identificar alrededor del 66% de todos los fraudes que ocurrieron. Esto es una métrica clave para no dejar pasar fraudes importantes.

## 4. F1-Score: 0.7574
**¿Qué es?**  
El F1-Score es una medida que combina la Precisión y el Recall en un solo número. Es útil cuando las dos métricas son importantes y no quieres sacrificar una por la otra. Es un promedio "balanceado".

**Interpretación:**  
Un F1-Score de 0.7574 (o 75.74%) sugiere un buen equilibrio entre la Precisión y el Recall. El modelo no es perfecto, pero hace un trabajo decente tanto identificando fraudes de forma correcta como detectando una buena parte de ellos.

## Resumen General del Rendimiento del Modelo
Tu modelo de detección de fraude simulado está mostrando un rendimiento prometedor:

- Es bastante preciso cuando identifica un fraude, lo que minimiza las interrupciones a los clientes por falsas alarmas.
- Es razonablemente bueno para detectar los fraudes existentes, aunque todavía hay un porcentaje de fraudes reales que podría no capturar (aproximadamente un 34%).
- El F1-Score confirma que hay un buen equilibrio general en su capacidad de detección.

**Nota importante:**  
Para un sistema de detección de fraude real, las empresas a menudo deben decidir si priorizan la precisión (no molestar a los clientes) o el recall (no dejar pasar ningún fraude), dependiendo del costo y la tolerancia al riesgo de cada tipo de error.